In [2]:
import os
import numpy as np

import shutil
from PIL import Image
import cv2 as cv
import random

import matplotlib.pyplot as plt

In [3]:
ROOT_DIR = os.getcwd()
ROOT_DIR

'C:\\Users\\HK Joshi\\Documents\\Github\\major_project\\biocube'

In [4]:
data_path = os.path.join(ROOT_DIR, 'datasets', 'VERA-fingervein')
cropped_data_path = os.path.join(ROOT_DIR, 'datasets',  'cropped-VERA-fingervein')
new_data_path = os.path.join(ROOT_DIR, 'datasets',  'LR-VERA-fingervein')

## 1. Crop images 50 pixels from top, left and bottom to remove bachground info

In [5]:
os.chdir(ROOT_DIR)
os.chdir(data_path)

for folder in os.listdir():
    if folder == 'METADATA.ods':
            continue
    os.chdir(folder)
    for file_name in os.listdir():
        img = np.array(Image.open(file_name).convert("RGB"))
        new_img = Image.fromarray(img[50:-51, 50: ,:]) #[h, w, c]
#         plt.imshow(img)
#         plt.show()
#         plt.imshow(new_img)
#         plt.show()
        
        new_file_path = os.path.join(cropped_data_path, folder)
        if os.path.isdir(new_file_path) == False:
            os.makedirs(new_file_path)
        dest = os.path.join(new_file_path)
        new_img.save(os.path.join(dest, f"{file_name.split('.')[0]}.{file_name.split('.')[1]}"))
    os.chdir('..')

# 2. Separate Left and Right fingervein dataset into different class

In [6]:
os.chdir(ROOT_DIR)
os.chdir(cropped_data_path)

for folder in os.listdir():
    # print(folder)
    if folder == 'METADATA.ods':
            continue
    os.chdir(folder)
    for file_name in os.listdir():
        if file_name.split('_')[1] == 'L':
            new_path = os.path.join(new_data_path, folder+'-L')
            if os.path.isdir(new_path) == False:
                os.makedirs(new_path)
            dest = os.path.join(new_path)
            shutil.copy2(file_name, dest)
        else:
            new_path = os.path.join(new_data_path, folder+'-R')
            if os.path.isdir(new_path) == False:
                os.makedirs(new_path)
            dest = os.path.join(new_path)
            shutil.copy2(file_name, dest)

    os.chdir('..')
os.chdir(ROOT_DIR)

# 3. Read images from a folder and rotate it for data augmentation

There are 2 images in a folder. Read the two images, take a random number between [-7, 7] that represents angle in degree. Rotate each image by given angle and save it. The new image name will be: "\<old_img_name\>rot\<degree\>.\<image_format\>"

In [7]:
degree_to_turn = 7

def gen_rot_img(img_lst):
    [path1, img1], [path2, img2] = img_lst
#     print(path1, img1, path2, img2)
    while(True): #Find the two values of angle such that none of them is zero
        deg1 = random.randint((-1 * degree_to_turn), degree_to_turn)
        deg2 = random.randint((-1 * degree_to_turn), degree_to_turn)
        # Return two degree angles who are different and not 0
        if (deg1 != 0 or deg2 != 0) and (deg1 != deg2):
            break
            
    rot_img11, rot_img12 = img1.rotate(deg1), img1.rotate(deg2)
    rot_img21, rot_img22 = img2.rotate(deg1), img2.rotate(deg2)
    
    rot_img11.save(f"{path1.split('.')[0]}rot{deg1}.{path1.split('.')[1]}"), rot_img12.save(f"{path1.split('.')[0]}rot{deg2}.{path1.split('.')[1]}")
    rot_img21.save(f"{path2.split('.')[0]}rot{deg1}.{path2.split('.')[1]}"), rot_img22.save(f"{path2.split('.')[0]}rot{deg2}.{path2.split('.')[1]}")
#     plt.subplot(141)
#     plt.imshow(rot_img11)
#     plt.subplot(142)
#     plt.imshow(rot_img12)
#     plt.subplot(143)
#     plt.imshow(rot_img21)
#     plt.subplot(144)
#     plt.imshow(rot_img22)
    

In [8]:
os.chdir(ROOT_DIR)
os.chdir(new_data_path)

for folder in os.listdir():
    files_in_folder = []
    os.chdir(folder)
    
    # Read the original two images in the folder
    for file in os.listdir():
        img = Image.open(file).convert("RGB")
        files_in_folder.append([file, img])
    
    # Create augmented images for both the images
    gen_rot_img(files_in_folder)
    
    os.chdir('..')
os.chdir(new_data_path)